In [ ]:
import os

def restart_runtime():
    os.kill(os.getpid(), 9)
    
# !pip3 install --user --extra-index-url https://<username>:<password>@pypi.netsquid.org netsquid
# restart_runtime()

In [ ]:
import netsquid as ns

In [ ]:
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.qubits.state_sampler import StateSampler
from netsquid.components import QuantumChannel
from netsquid.components.models import FixedDelayModel
import netsquid.qubits.ketstates as ks
from netsquid.qubits import qubitapi as qapi


from netsquid.components import QuantumChannel
from netsquid.components.models import DepolarNoiseModel, FibreLossModel
from netsquid.components.models import FibreDelayModel

from netsquid.nodes.connections import Connection
from netsquid.components import ClassicalChannel
from netsquid.components.models import FibreDelayModel

import netsquid.components.instructions as instr
from netsquid.components.qprocessor import QuantumProcessor
from netsquid.components.models.qerrormodels import DephaseNoiseModel, DepolarNoiseModel
from netsquid.components.qprocessor import PhysicalInstruction

In [ ]:
ns.set_qstate_formalism(ns.QFormalism.DM)

In [ ]:
class ClassicalConnection(Connection):
    def __init__(self, length, name="ClassicalConnection"):
        super().__init__(name=name)
        # forward A Port to ClassicalChannel send Port
        # forward ClassicalChannel recv Port to B Port
        self.add_subcomponent(ClassicalChannel("Channel_A2B", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])

In [ ]:
class QuantumConnection(Connection):
    def __init__(self, length, depolar_rate, loss_enabled):
        
        super().__init__(name="QuantumConnection")

        if loss_enabled:
          models={"delay_model": FibreDelayModel(),
                    'quantum_noise_model' : DepolarNoiseModel(depolar_rate=depolar_rate),
                    'quantum_loss_model' : FibreLossModel()}
        else:
          models={"delay_model": FibreDelayModel(),
                    'quantum_noise_model' : DepolarNoiseModel(depolar_rate=depolar_rate)}

        self.add_subcomponent(QuantumChannel("qChannel_A2B", length=length,
                              models = models),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])

In [ ]:
def create_processor(measure_noise_rate, memory_noise_rate):
    """Factory to create a quantum processor.

    Has 2 memory positions and the physical instructions necessary
    for teleportation.

    Parameters
    ----------
    measure_noise_rate : float
        Dephase rate of qubits measurement
    memory_noise_rate : float
        Depolar rate of qubit idling.

    Returns
    -------
    :class:`~netsquid.components.qprocessor.QuantumProcessor`
        A quantum processor to specification.

    """
    measure_noise_model = DephaseNoiseModel(dephase_rate=measure_noise_rate,
                                            time_independent=True)
    physical_instructions = [
        
        PhysicalInstruction(instr.INSTR_INIT, duration=1, parallel=True),

        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True, topology=[0,1]),

        PhysicalInstruction(instr.INSTR_X, duration=1, parallel=True, topology=[0]),

        PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True, topology=[0]),

        PhysicalInstruction(instr.INSTR_S, duration=1, parallel=True, topology=[0]),

        PhysicalInstruction(instr.INSTR_CNOT, duration=4, topology=[(0, 1)]),

        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, topology=[0],
                            quantum_noise_model=measure_noise_model),

        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, topology=[1],
                            quantum_noise_model=measure_noise_model)

    ]
    memory_noise_model = DepolarNoiseModel(depolar_rate=memory_noise_rate)

    processor = QuantumProcessor("quantum_processor", num_positions=2,
                                 memory_noise_models=[memory_noise_model]*2,
                                 phys_instructions=physical_instructions)

    return processor

In [ ]:
from netsquid.nodes import Node
from netsquid.nodes import Network

def network_setup(node_distance=4e-3, memory_noise_rate=5e4, measure_noise_rate=0.3, link_noise_rate=0, loss_enabled=True):
    """Setup the physical components of the quantum network.

    Parameters
    ----------
    node_distance : float, optional
        Distance between nodes.
    memory_noise_rate : float, optional
        Depolarization rate of qubits idling in memory.
    measure_noise_rate : float, optional
        Dephasing rate of physical measurement.
    link_noise_rate : float, optional
        Depolarization rate of qubits across channels
    loss_enabled : bool, optional
        Enables photon attenuation across channels

    Returns
    -------
    :class:`~netsquid.nodes.node.Network`

    """
    alice = Node("Alice", qmemory=create_processor(measure_noise_rate, memory_noise_rate))

    bob =  Node("Bob", qmemory=create_processor(measure_noise_rate, memory_noise_rate))

    bp_source = Node("bp_source")

    qsource = QSource(f"qsource_bp_source", StateSampler([ks.b00], [1.0]), num_ports=2,
                          status=SourceStatus.EXTERNAL)
    bp_source.add_subcomponent(qsource, name="qsource")

    network = Network("Quantum Teleportation Network")
    network.add_nodes([alice, bob, bp_source])

    c_conn_a2bps = ClassicalConnection(length=node_distance)

    network.add_connection(alice, bp_source, connection=c_conn_a2bps, label="c_conn_a2bps",
                           port_name_node1="cout_a2bps", port_name_node2="cin_bpsFa")

    bp_source.ports["cin_bpsFa"].forward_input(bp_source.subcomponents["qsource"].ports["trigger"])

    c_conn_a2b = ClassicalConnection(length=node_distance)

    network.add_connection(alice, bob, connection=c_conn_a2b, label="c_conn_a2b",
                           port_name_node1="cout_a2b", port_name_node2="cin_bFa")

    c_conn_b2a = ClassicalConnection(length=node_distance)

    network.add_connection(bob, alice, connection=c_conn_b2a, label="c_conn_b2a",
                           port_name_node1="cout_b2a", port_name_node2="cin_aFb")

    q_conn_bps2a = QuantumConnection(length=node_distance, depolar_rate=link_noise_rate, loss_enabled=loss_enabled)

    qport_bps2a, q_port_aFbps = network.add_connection(bp_source, alice, connection=q_conn_bps2a,
                                                     label="q_conn_bps2a",
                                                     port_name_node1="qout_bps2a",
                                                     port_name_node2="qin_aFbps")

    bp_source.subcomponents["qsource"].ports["qout0"].forward_output(bp_source.ports["qout_bps2a"])

    alice.ports["qin_aFbps"].forward_input(alice.qmemory.ports["qin1"])

    q_conn_bps2b = QuantumConnection(length=node_distance, depolar_rate=link_noise_rate, loss_enabled=False)

    qport_bps2b, q_port_bFbps = network.add_connection(bp_source, bob, connection=q_conn_bps2b,
                                                     label="q_conn_bps2b",
                                                     port_name_node1="qout_bps2b",
                                                     port_name_node2="qin_bFbps")

    bp_source.subcomponents["qsource"].ports["qout1"].forward_output(bp_source.ports["qout_bps2b"])

    bob.ports["qin_bFbps"].forward_input(bob.qmemory.ports["qin0"])

    return network

In [ ]:
from netsquid.components.qprogram import QuantumProgram

class InitQubitXProgram(QuantumProgram):
    """Program to create a qubit and transform it to the y0 state.

    """
    default_num_qubits = 1

    def program(self):
        q1, = self.get_qubit_indices(1)

        self.apply(instr.INSTR_INIT, q1)

        self.apply(instr.INSTR_H, q1)

        self.apply(instr.INSTR_S, q1)

        yield self.run()

In [ ]:
class BellMeasurementProgram(QuantumProgram):
    """Program to perform a Bell measurement on two qubits.

    Measurement results are stored in output keys "M1" and "M2"

    """
    default_num_qubits = 2

    def program(self):
        q1, q2 = self.get_qubit_indices(2)

        self.apply(instr.INSTR_CNOT, [q1, q2])
        self.apply(instr.INSTR_H, q1)
        self.apply(instr.INSTR_MEASURE, q1, output_key="M1")
        self.apply(instr.INSTR_MEASURE, q2, output_key="M2")

        yield self.run()